# 💬 AgRAG

Este notebook configura um pipeline completo de perguntas e respostas (Q&A) com base em um arquivo PDF, usando RAG (Retrieval-Augmented Generation), FAISS, embeddings com HuggingFace, e LLM Gemma via Ollama.

In [ ]:
# 📌 Instalação dos pacotes necessários
%pip install langchain langchain-ollama langchain-huggingface langchain-community faiss-cpu pypdf sentence-transformers ollama pdfplumber

In [8]:
import pdfplumber

def load_pdf_text(path):
    all_text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            all_text += page.extract_text() + "\n"
    return all_text

pdf_text = load_pdf_text("data/500perguntassoja.pdf")
print(pdf_text[:1000])


Coleção500 Perguntas500 Respostas
O produtor pergunta, a Embrapa responde

Empresa Brasileira de Pesquisa Agropecuária
Embrapa Soja
Ministério da Agricultura, Pecuária e Abastecimento
O produtor pergunta, a Embrapa responde
Arnold Barbosa de Oliveira
Regina Maria Villas Bôas de Campos Leite
Alvadi Antônio Balbinot Junior
Claudine Dinali Santos Seixas
Hugo Soares Kern
Editores Técnicos
Embrapa
Brasília, DF
2019
Exemplares desta publicação podem ser adquiridos na:
Embrapa Responsável pela edição
Parque Estação Biológica (PqEB) Secretaria-Geral
Av. W3 Norte (final)
Coordenação editorial
70770-901 Brasília, DF
Alexandre de Oliveira Barcellos
Fone: (61) 3448-4236
Heloiza Dias da Silva
Fax: (61) 3448-2494
Nilda Maria da Cunha Sette
www.embrapa.br/livraria
livraria@embrapa.br Supervisão editorial
Embrapa Soja Josmária Madalena Lopes
Rod. Carlos João Strass, acesso Orlando Amaral,
Revisão de texto
Distrito de Warta
Jane Baptistone de Araújo
Caixa Postal 231
86001-970 Londrina, PR
Normalizaç

In [9]:
# 📌 Quebra de texto e geração dos embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
docs = splitter.create_documents([pdf_text])

embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
db = FAISS.from_documents(docs, embedding)

In [10]:
# 📌 Configuração do modelo Gemma via Ollama
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gemma3:1b")  # Requer que o modelo esteja instalado via `ollama pull gemma3:1b`

In [11]:
# 📌 Criação do chain de perguntas e respostas (RAG)
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

crop = "soja"  # Ajuste para a cultura desejada

template = """
Você é um assistente agrícola especializado na cultura de """ + crop + """ . Responda a pergunta abaixo em português do Brasil, com base exclusivamente no conteúdo fornecido.

Contexto:
{context}

Pergunta: {question}
Resposta:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
)

In [12]:
# 📌 Fazer perguntas ao modelo
pergunta = "Qual importância do controle biológico na soja?"
resposta = qa_chain.invoke(pergunta)
print("Resposta:", resposta["result"])

Resposta: O controle biológico na soja é de extrema importância por diversas razões:

*   **Autossustentabilidade:** Os agentes de controle biológico são considerados inócuos a outros organismos não alvo e não provocam desequilíbrio no ambiente, o que o torna uma alternativa mais amigável ao controle químico.

*   **Resistência:** A utilização de agentes biológicos pode ajudar a mitigar a disseminação de pragas resistentes a inseticidas, pois eles atuam de forma natural e complementam os agentes químicos.

*   **Diversidade:** O controle biológico promove a diversidade de organismos no solo, o que aumenta a resiliência da cultura e reduz a dependência de produtos químicos.

*   **Alternativa:** Em situações de emergência, o controle biológico pode ser utilizado para controlar pragas de forma rápida e eficiente.

*   **Benefícios Ambientais:** Além de combater pragas, o controle biológico contribui para a saúde do solo e da água.

*   **Redução de Custos:** Em alguns casos, o controle b